In [13]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pandas as pd

In [14]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [17]:
product_df = pd.read_csv('C:/Users/ACER/Downloads/Amazon-Products.csv')
product_name = product_df['name']
product_name = product_name.str.replace('''[^\w\d\s]''','',regex=True).str.lower()
product_name = product_name.drop_duplicates()
product_name = product_name.str.lower()
product_name = product_name.tolist()

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
C:\Users\ACER\AppData\Local\Temp\ipykernel_17448\3381987994.py:3: SyntaxWarning: invalid escape sequence '\w'
  product_name = product_name.str.replace('''[^\w\d\s]''','',regex=True).str.lower()


In [4]:
count = product_df['name'].str.endswith('...', na=False).sum()
print(count)

122226


In [15]:
print(len(product_df))

551585


In [9]:

batch_size = 1000
embedding_file = "../data/embeddings.npy"
num_products = len(product_name)
# Generate embeddings in batches and save them
all_embeddings = []
for i in range(0, num_products, batch_size):
    batch_texts = product_name[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts).astype('float32')
    all_embeddings.append(batch_embeddings)

all_embeddings = np.vstack(all_embeddings)
np.save(embedding_file, all_embeddings)


In [ ]:
all_embeddings = np.load('../model/embeddings.npy')
dimension = all_embeddings.shape[1]

nlist = 100  
quantizer = faiss.IndexFlatL2(dimension) 
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

index.train(all_embeddings)

batch_size = 10000
for i in range(0, all_embeddings.shape[0], batch_size):
    batch_embeddings = all_embeddings[i:i+batch_size]
    index.add(batch_embeddings)
    
# Save the index to disk
faiss.write_index(index, "../data/large_index.ivf")

In [18]:
index = faiss.read_index("../model/large_index.ivf")

# Query example
query = "iphone"
query_embedding = model.encode([query]).astype('float32')

# Perform similarity search
k = 20  # Top 10 results
distances, indices = index.search(query_embedding, k)

# Display results
print("Query:", query)
for i in range(k):
    print(f"Result {i+1}:")
    print(f"  Text: {product_name[indices[0][i]]}")
    print(f"  Distance: {distances[0][i]}")

Query: iphone
Result 1:
  Text: apple iphone 14 plus 256 gb  blue
  Distance: 0.8725134134292603
Result 2:
  Text: apple iphone 13 256gb  midnight
  Distance: 0.8844698667526245
Result 3:
  Text: apple iphone 13 128gb  midnight
  Distance: 0.9230005741119385
Result 4:
  Text: apple iphone 12 64gb  white
  Distance: 0.9341723918914795
Result 5:
  Text: apple iphone 12 64gb  blue
  Distance: 0.9800862073898315
Result 6:
  Text: apple iphone 12 64gb  black
  Distance: 0.9819310903549194
Result 7:
  Text: apple iphone 14 pro 128 gb  deep purple
  Distance: 1.0029823780059814
Result 8:
  Text: apple iphone 13 256gb  blue
  Distance: 1.0047192573547363
Result 9:
  Text: apple iphone 12 128gb  black
  Distance: 1.018121600151062
Result 10:
  Text: apple iphone 14 128 gb  product red
  Distance: 1.0184566974639893
Result 11:
  Text: apple iphone 13 128gb  blue
  Distance: 1.0222928524017334
Result 12:
  Text: apple iphone 14 pro max 256 gb  deep purple
  Distance: 1.0301899909973145
Result 13: